## Preliminary Results

In [1]:
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np
import gspread
import warnings
warnings.filterwarnings("ignore")

---
### Connect to table

In [2]:
## Connect to our service account
project = 'API-Project-73a06f99e859.json'
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name(project, scope)
gc = gspread.authorize(credentials)

In [3]:
spreadsheet_key = '1d083PFuyFRXtweoyHNtHDuAIUsBZNlLKBlA9QWYLtfI'
book = gc.open_by_key(spreadsheet_key)

data = book.worksheet('web_extraction').get_all_values()
web_extraction = pd.DataFrame(data[1:], columns=data[0])
data = book.worksheet('manual_search').get_all_values()
manual_search = pd.DataFrame(data[1:], columns=data[0])
data = book.worksheet('external_sources').get_all_values()
external_sources = pd.DataFrame(data[1:], columns=data[0])
data = book.worksheet('questionnaires').get_all_values()
questionnaires = pd.DataFrame(data[1:], columns=data[0])

In [4]:
web_extraction = web_extraction[['name', 'url', 'description', 'topic', 'group', 'method', 'source',
       'source_type', 'relevant', 'small description']]

In [5]:
table = pd.DataFrame()
table = table.append(web_extraction)
table = table.append(manual_search)
table = table.append(external_sources)
table = table.append(questionnaires)

In [22]:
n1 = len(web_extraction[web_extraction['relevant']=='YES'])
n2 = len(manual_search[manual_search['relevant']=='YES'])
n3 = len(external_sources[external_sources['relevant']=='YES'])
n4 = len(questionnaires[questionnaires['relevant']=='YES'])
print('Total number of relevant tools: {}\n\nWeb extraction and sample open databases: {}\nManual extraction private databases: {}\nExternal Sources and desk search: {}\nConsumer agencies responses: {}'.format(n1+n2+n3+n4,n1, n2, n3, n4))


Total number of relevant tools: 156

Web extraction and sample open databases: 91
Manual extraction private databases: 15
External Sources and desk search: 30
Consumer agencies responses: 20


In [6]:
table.columns

Index(['name', 'url', 'description', 'topic', 'group', 'method', 'source',
       'source_type', 'relevant', 'small description'],
      dtype='object')

In [7]:
relevat_tools = table[table['relevant']=='YES']

In [8]:
len(relevat_tools)

156

In [12]:
relevat_tools\
    .groupby(['group','topic'])\
    .count()['url']\
    .reset_index()\
    .sort_values('url', ascending=False)\
    .reset_index(drop=True)\

,group,topic,url
0,Advertising,Behavioural tracking,20
1,Scam (Online Fraud),General scam & Pishing,18
2,Advertising,Fake reviews,15
3,Web Evidence,Internet snaps and internet archives,13
4,Web Evidence,Domain registries,11
5,Advertising,Misleading advertising,9
6,Advertising,Online trackers,8
7,Geoblocking,VPN,7
8,Advertising,Web beacon tracker,6
9,Compliance,Failure of Web accessibility,6


In [23]:
# pd.pivot_table(
#     relevat_tools,
#     values=None,
#     index='topic',
#     columns='source_type',
#     aggfunc='count',
#     fill_value=0,
#     margins=False,
#     dropna=True,
#     margins_name='All',
# )['url'].to_csv('table.csv')